In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from astropy.io import fits
import os
import datetime
from matplotlib.colors import LogNorm
from pybirales.birales import BiralesFacade
from pybirales.birales_config import BiralesConfig
from pybirales.pipeline.modules.detection.preprocessor import PreProcessor
from pybirales.pipeline.modules.detection.dbscan_detection import _validate, _create, detect, partition_input_data, dbscan_clustering,create_clusters 
from pybirales.pipeline.modules.detection.exceptions import DetectionClusterIsNotValid
from pybirales.pipeline.modules.detection.queue import BeamCandidatesQueue
from pybirales.pipeline.modules.detection.space_debris_candidate import SpaceDebrisTrack
from scipy import stats
from pybirales import settings

matplotlib.rcParams['figure.figsize'] = (12, 10)

In [2]:
# Global variables
HOME_DIR = os.environ['HOME']
FITS_DIR = os.path.join(HOME_DIR, '.birales/visualisation/fits/')
FITS_FILENAME = 'Observation_2018-04-11T1010/Observation_2018-04-11T1010_raw.fits'
BEAM_ID = 4
OBS_INFO = {
    'start_center_frequency':410.0703125,
    'channel_bandwidth':9.5367431640625e-06,
    'transmitter_frequency': 410.105,
    'timestamp': datetime.datetime(2017, 6, 30, 12, 13, 30, 455859),
    'sampling_time':0.1048576,
    'nchans': 8192,
}
BIRALES_CONFIG = os.path.join(HOME_DIR, '.birales/configuration/birales.ini')
DETECTION_CONFIG = os.path.join(HOME_DIR, '.birales/configuration/detection.ini')

In [3]:
# Load the BIRALES configuration from file
config = BiralesConfig([BIRALES_CONFIG, DETECTION_CONFIG], {})

# Initialise the Birales Facade (BOSS)
bf = BiralesFacade(configuration=config)



2018-04-12 10:36:17,216 INFO     26817 MainProcess     MainThread           Loading configuration files
2018-04-12 10:36:17,218 INFO     26817 MainProcess     MainThread           Loading the /home/denis/.birales/configuration/birales.ini configuration file.
2018-04-12 10:36:17,223 INFO     26817 MainProcess     MainThread           Loaded the /home/denis/.birales/configuration/birales.ini configuration file.
2018-04-12 10:36:17,225 INFO     26817 MainProcess     MainThread           Loading the /home/denis/.birales/configuration/detection.ini configuration file.
2018-04-12 10:36:17,227 INFO     26817 MainProcess     MainThread           Loaded the /home/denis/.birales/configuration/detection.ini configuration file.
2018-04-12 10:36:17,229 INFO     26817 MainProcess     MainThread           Loading the /home/denis/.venv/birales/local/lib/python2.7/site-packages/pybirales/configuration/backend/roach_backend.ini configuration file.
2018-04-12 10:36:17,231 INFO     26817 MainProcess     M

In [4]:
raw_fits_filepath = os.path.join(FITS_DIR, FITS_FILENAME)
input_data = fits.getdata(raw_fits_filepath)
print ('Filtered image shape:', input_data.shape)
print settings.detection.n_noise_samples


('Filtered image shape:', (1, 8192, 288))
5


In [22]:
# Get the first N blobs of data, on which the noise estimate will be calculated
# Skip the first two blobs
data = input_data[:,:, 32*2:32 * settings.detection.n_noise_samples]
data.shape

(1, 8192, 96)

In [29]:
np.sqrt(np.mean(np.power(data, 2.0), axis=2)).shape

(1, 8192)

In [76]:
power = np.power(np.abs(data), 2.0)
power.shape

rms_noise = np.sqrt(np.mean(power, axis=2))
rms_noise.shape

(1, 8192)

In [80]:
# Determine the Beam Channel Noise
rms_noise2 = rms_noise.copy()
rms_noise2 *= 1.2

noise = np.empty(shape=(1, 8192, 5))
noise[:] = np.nan

noise[:, :, 0] = rms_noise
noise[:, :, 1] = rms_noise2

beam_noise = np.nanmean(noise, axis=2)
beam_noise.shape

(1, 8192)

In [94]:
# Test SNR division
snr = input_data / beam_noise[:, :, np.newaxis]

beam_id = 0
test_channel = 11
test_time = 12
expected = input_data[beam_id, test_channel, test_time]/ beam_noise[0,test_channel]
actual = snr[beam_id, test_channel, test_time]

print(beam_noise[0,test_channel], expected, actual, expected == actual)


(33.53048096410884, 2.7315441376022727, 2.7315441376022727, True)


In [95]:
# Noise statistics
print (np.min(beam_noise), np.max(beam_noise), np.mean(beam_noise))

(10.22626071254055, 46.54869159639895, 24.260943132641238)
